# 信息论
:label:`sec_information_theory`

宇宙中充满了信息。信息提供了一种跨越学科鸿沟的通用语言：从莎士比亚的十四行诗到Cornell ArXiv上的研究论文，从梵高的《星夜》到贝多芬的《第五交响曲》，从第一种编程语言Plankalkül到最先进的机器学习算法。无论格式如何，一切都必须遵循信息论的规则。通过信息论，我们可以测量和比较不同信号中的信息量。在本节中，我们将探讨信息论的基本概念及其在机器学习中的应用。

在开始之前，让我们概述一下机器学习与信息论之间的关系。机器学习旨在从数据中提取有趣的信号并进行关键预测。另一方面，信息论研究信息的编码、解码、传输和处理。因此，信息论为讨论机器学习系统中的信息处理提供了基本语言。例如，许多机器学习应用使用如:numref:`sec_softmax`中描述的交叉熵损失。这种损失可以直接从信息理论考虑中推导出来。

## 信息

让我们从信息论的“灵魂”开始：信息。*信息*可以以一种或多种编码格式的特定序列编码在任何事物中。假设我们任务是尝试定义信息的概念。我们的起点可能是什么？

考虑以下思想实验。我们有一个朋友有一副扑克牌。他们将洗牌，翻开一些牌，并告诉我们关于这些牌的陈述。我们将尝试评估每个陈述的信息内容。

首先，他们翻开一张牌并说，“我看到一张牌。”这根本没有提供给我们任何信息。我们已经确定这是事实，所以我们希望信息量应该是零。

接下来，他们翻开一张牌并说，“我看到一个红心。”这提供了一些信息，但实际上只有4种不同的花色，每种花色的可能性相同，所以这个结果并没有让我们感到惊讶。我们希望无论信息的度量如何，这个事件的信息含量应该很低。

接下来，他们翻开一张牌并说，“这是黑桃3。”这是更多的信息。确实有52种可能性，每种可能性都相等，而我们的朋友告诉了我们具体是哪一种。这应该是一个中等数量的信息。

让我们把这推向逻辑极端。假设最后他们翻开整副牌并读出洗牌后的整个序列。牌的顺序有52!种不同的排列，同样每种可能性都是相等的，所以我们需要很多信息来知道具体是哪一种。

我们发展的任何信息概念都必须符合这种直觉。实际上，在接下来的部分中，我们将学习如何计算这些事件分别具有$0\textrm{ bits}$, $2\textrm{ bits}$, $~5.7\textrm{ bits}$, 和 $~225.6\textrm{ bits}$的信息。

如果我们仔细阅读这些思想实验，我们会看到一个自然的想法。作为一个起点，与其关心知识本身，我们可能会基于信息代表事件的惊奇程度或抽象可能性的想法。例如，如果我们要描述一个不寻常的事件，我们需要大量的信息。对于一个常见的事件，我们可能不需要太多信息。

1948年，Claude E. Shannon发表了《通信的数学理论》:cite:`Shannon.1948`，建立了信息论。在他的文章中，Shannon首次引入了信息熵的概念。我们将从这里开始我们的旅程。

### 自信息

由于信息体现了事件的抽象可能性，我们如何将可能性映射到比特数？Shannon引入了*比特*作为信息单位，这个词最初是由John Tukey创造的。那么什么是“比特”，为什么我们用它来衡量信息？历史上，古董发射机只能发送或接收两种类型的代码：$0$和$1$。事实上，二进制编码仍然在所有现代数字计算机中普遍使用。这样，任何信息都由一系列$0$和$1$编码。因此，长度为$n$的一系列二进制数字包含$n$比特的信息。

现在，假设对于任何一系列代码，每个$0$或$1$出现的概率为$\frac{1}{2}$。因此，一个事件$X$具有一系列长度为$n$的代码，发生的概率为$\frac{1}{2^n}$。同时，正如我们前面提到的，这一系列包含$n$比特的信息。那么，我们能否推广到一个数学函数，该函数可以将概率$p$转换为比特数？Shannon通过定义*自信息*

$$I(X) = - \log_2 (p),$$

作为我们收到的此事件$X$的*比特*信息给出了答案。请注意，在本节中我们将始终使用以2为底的对数。为了简单起见，本节其余部分将省略对数符号中的下标2，即$\log(.)$始终指的是$\log_2(.)$。例如，代码"0010"的自信息

$$I(\textrm{"0010"}) = - \log (p(\textrm{"0010"})) = - \log \left( \frac{1}{2^4} \right) = 4 \textrm{ bits}.$$

我们可以如下所示计算自信息。在此之前，让我们先导入本节所需的所有必要包。

In [1]:
import torch
from torch.nn import NLLLoss


def nansum(x):
    # Define nansum, as pytorch does not offer it inbuilt.
    return x[~torch.isnan(x)].sum()

def self_information(p):
    return -torch.log2(torch.tensor(p)).item()

self_information(1 / 64)

6.0

## 熵

由于自信息仅衡量单个离散事件的信息，我们需要一个更通用的度量来处理任何具有离散或连续分布的随机变量。

### 熵的动机

让我们具体说明我们想要什么。这将是关于香农熵公理的非正式陈述。事实证明，以下这些常识性陈述迫使我们对信息有一个独特的定义。在 :citet:`Csiszar.2008` 中可以找到这些公理的形式版本以及其他几个公理。

1. 通过观察随机变量获得的信息不依赖于我们如何称呼元素，或者存在概率为零的额外元素。
2. 通过观察两个随机变量获得的信息不超过分别观察它们所获得信息的总和。如果它们是独立的，则正好是总和。
3. 观察（几乎）确定事件时获得的信息（几乎）为零。

虽然证明这一事实超出了本文的范围，但重要的是要知道这一点唯一地决定了熵必须采取的形式。这些公理允许的唯一不确定性在于基本单位的选择，通常通过选择我们之前看到的单次公平硬币翻转提供的信息为一个比特来进行标准化。

### 定义

对于遵循概率分布 $P$ 并具有概率密度函数 (p.d.f.) 或概率质量函数 (p.m.f.) $p(x)$ 的任何随机变量 $X$，我们通过*熵*（或*香农熵*）测量预期信息量

$$H(X) = - E_{x \sim P} [\log p(x)].$$
:eqlabel:`eq_ent_def`

具体来说，如果 $X$ 是离散的，$$H(X) = - \sum_i p_i \log p_i \textrm{, 其中 } p_i = P(X_i).$$

否则，如果 $X$ 是连续的，我们也称熵为*微分熵*

$$H(X) = - \int_x p(x) \log p(x) \; dx.$$

In [2]:
def entropy(p):
    entropy = - p * torch.log2(p)
    # Operator `nansum` will sum up the non-nan number
    out = nansum(entropy)
    return out

entropy(torch.tensor([0.1, 0.5, 0.1, 0.3]))

tensor(1.6855)

### 解释

你可能好奇：在熵的定义 :eqref:`eq_ent_def` 中，为什么我们使用负对数的期望值？这里有一些直觉。

首先，为什么使用 *对数* 函数 $\log$？假设 $p(x) = f_1(x) f_2(x) \ldots, f_n(x)$，其中每个组成部分函数 $f_i(x)$ 之间是独立的。这意味着每个 $f_i(x)$ 独立地为从 $p(x)$ 获得的总信息做出贡献。如上所述，我们希望熵公式在独立随机变量上是可加的。幸运的是，$\log$ 可以自然地将概率分布的乘积转换为各个项的求和。

其次，为什么使用 *负* 对数？直观地说，更频繁的事件应该比不常见的事件包含的信息少，因为我们通常从不寻常的情况中获得的信息比从普通情况中获得的多。然而，$\log$ 随着概率的增加而单调递增，并且在 $[0, 1]$ 区间内确实为负。我们需要构建一个概率与熵之间的单调递减关系，这理想上应该是始终为正的（因为我们观察到的任何东西都不应迫使我们忘记已知的东西）。因此，我们在 $\log$ 函数前加上一个负号。

最后，*期望* 函数是从哪里来的？考虑一个随机变量 $X$。我们可以将自信息 ($-\log(p)$) 解释为我们看到特定结果时所感到的 *惊讶* 程度。实际上，当概率接近零时，惊讶程度变得无限大。同样，我们可以将熵解释为观察 $X$ 时的平均惊讶程度。例如，想象一个老虎机系统以统计独立的方式发出符号 ${s_1, \ldots, s_k}$，其概率分别为 ${p_1, \ldots, p_k}$。那么这个系统的熵等于观察每个输出的平均自信息，即

$$H(S) = \sum_i {p_i \cdot I(s_i)} = - \sum_i {p_i \cdot \log p_i}.$$



### 熵的性质

通过上述例子和解释，我们可以推导出熵 :eqref:`eq_ent_def` 的以下性质。在这里，我们将 $X$ 称为事件，将 $P$ 称为 $X$ 的概率分布。

* 对于所有离散的 $X$，$H(X) \geq 0$（对于连续的 $X$，熵可以为负）。

* 如果 $X \sim P$ 具有概率密度函数或概率质量函数 $p(x)$，并且我们尝试用一个新的概率分布 $Q$ 来估计 $P$，该分布具有概率密度函数或概率质量函数 $q(x)$，则 $$H(X) = - E_{x \sim P} [\log p(x)] \leq  - E_{x \sim P} [\log q(x)], \textrm{ 当且仅当 } P = Q \textrm{ 时等式成立。}$$ 另外，$H(X)$ 给出了从 $P$ 中抽取符号所需的平均比特数的下界。

* 如果 $X \sim P$，那么当它均匀分布在所有可能的结果中时，$x$ 传递的信息量最大。具体来说，如果概率分布 $P$ 是离散的，具有 $k$ 类 $\{p_1, \ldots, p_k \}$，则 $$H(X) \leq \log(k), \textrm{ 当且仅当 } p_i = \frac{1}{k}, \forall i \textrm{ 时等式成立。}$$ 如果 $P$ 是一个连续随机变量，则情况会复杂得多。然而，如果我们进一步要求 $P$ 支持在一个有限区间上（所有值介于 $0$ 和 $1$ 之间），则当 $P$ 在该区间上是均匀分布时，$P$ 具有最高的熵。


## 互信息

之前我们定义了单个随机变量 $X$ 的熵，那么一对随机变量 $(X, Y)$ 的熵呢？我们可以将这些技术视为试图回答以下类型的问题，“$X$ 和 $Y$ 一起包含的信息与它们各自包含的信息相比如何？是否有冗余信息，还是都是独特的？”

在接下来的讨论中，我们总是将 $(X, Y)$ 视为遵循联合概率分布 $P$ 的一对随机变量，其概率密度函数或概率质量函数为 $p_{X, Y}(x, y)$，而 $X$ 和 $Y$ 分别遵循概率分布 $p_X(x)$ 和 $p_Y(y)$。


### 联合熵

类似于单个随机变量的熵 :eqref:`eq_ent_def`，我们定义一对随机变量 $(X, Y)$ 的 *联合熵* $H(X, Y)$ 为

$$H(X, Y) = -E_{(x, y) \sim P} [\log p_{X, Y}(x, y)]. $$
:eqlabel:`eq_joint_ent_def`

确切地说，一方面，如果 $(X, Y)$ 是一对离散随机变量，则

$$H(X, Y) = - \sum_{x} \sum_{y} p_{X, Y}(x, y) \log p_{X, Y}(x, y).$$

另一方面，如果 $(X, Y)$ 是一对连续随机变量，则我们定义 *微分联合熵* 为

$$H(X, Y) = - \int_{x, y} p_{X, Y}(x, y) \ \log p_{X, Y}(x, y) \;dx \;dy.$$

我们可以认为 :eqref:`eq_joint_ent_def` 告诉我们这对随机变量中的总随机性。作为两个极端，如果 $X = Y$ 是两个相同的随机变量，则这对中的信息正好是一个中的信息，我们有 $H(X, Y) = H(X) = H(Y)$。另一个极端，如果 $X$ 和 $Y$ 是独立的，则 $H(X, Y) = H(X) + H(Y)$。实际上，我们总是发现一对随机变量中的信息量不会小于任一随机变量的熵，也不会大于两者之和。

$$
H(X), H(Y) \le H(X, Y) \le H(X) + H(Y).
$$

让我们从头开始实现联合熵。

In [3]:
def joint_entropy(p_xy):
    joint_ent = -p_xy * torch.log2(p_xy)
    # Operator `nansum` will sum up the non-nan number
    out = nansum(joint_ent)
    return out

joint_entropy(torch.tensor([[0.1, 0.5], [0.1, 0.3]]))

tensor(1.6855)

请注意，这与之前的*代码*相同，但现在我们将其解释为对两个随机变量的联合分布进行操作。

### 条件熵

上面定义的联合熵表示一对随机变量中包含的信息量。这是有用的，但通常这不是我们关心的内容。考虑机器学习的场景。设$X$是描述图像像素值的随机变量（或随机变量向量），$Y$是类别标签的随机变量。$X$应该包含大量信息——自然图像是复杂的事物。然而，一旦显示了图像，$Y$所含的信息量应该是低的。实际上，除非数字难以辨认，否则数字的图像应已经包含了关于它是哪个数字的信息。因此，为了继续扩展我们的信息理论词汇，我们需要能够推理一个随机变量在另一个条件下的信息内容。

在概率论中，我们看到了*条件概率*的定义来衡量变量之间的关系。现在我们要类似地定义*条件熵*$H(Y \mid X)$。我们可以写作

$$ H(Y \mid X) = - E_{(x, y) \sim P} [\log p(y \mid x)],$$
:eqlabel:`eq_cond_ent_def`

其中$p(y \mid x) = \frac{p_{X, Y}(x, y)}{p_X(x)}$是条件概率。具体来说，如果$(X, Y)$是一对离散随机变量，则

$$H(Y \mid X) = - \sum_{x} \sum_{y} p(x, y) \log p(y \mid x).$$

如果$(X, Y)$是一对连续随机变量，则*微分条件熵*同样定义为

$$H(Y \mid X) = - \int_x \int_y p(x, y) \ \log p(y \mid x) \;dx \;dy.$$


现在很自然地会问，*条件熵*$H(Y \mid X)$是如何与熵$H(X)$和联合熵$H(X, Y)$相关的？利用上述定义，我们可以简洁地表达这一点：

$$H(Y \mid X) = H(X, Y) - H(X).$$

这有一个直观的解释：给定$X$时$Y$中的信息($H(Y \mid X)$)等于$X$和$Y$一起的信息($H(X, Y)$)减去$X$中已有的信息。这就给出了$Y$中未被$X$代表的信息。

现在，让我们从零开始实现条件熵 :eqref:`eq_cond_ent_def`。

In [4]:
def conditional_entropy(p_xy, p_x):
    p_y_given_x = p_xy/p_x
    cond_ent = -p_xy * torch.log2(p_y_given_x)
    # Operator `nansum` will sum up the non-nan number
    out = nansum(cond_ent)
    return out

conditional_entropy(torch.tensor([[0.1, 0.5], [0.2, 0.3]]),
                    torch.tensor([0.2, 0.8]))

tensor(0.8635)

### 互信息

给定随机变量 $(X, Y)$ 的先前设定，你可能会想：“既然我们知道 $Y$ 中包含但 $X$ 中没有的信息量，我们是否可以类似地问 $X$ 和 $Y$ 之间共享了多少信息？”答案就是 $(X, Y)$ 的*互信息*，我们将它写作 $I(X, Y)$。

与其直接进入正式定义，不如先通过尝试完全基于我们之前构建的术语来推导互信息的表达式来练习我们的直觉。我们希望找到两个随机变量之间的共享信息。我们可以尝试的一种方法是从 $X$ 和 $Y$ 共同包含的所有信息开始，然后减去那些不共享的部分。$X$ 和 $Y$ 共同包含的信息写为 $H(X, Y)$。我们想要从中减去的是 $X$ 中包含但 $Y$ 中没有的信息，以及 $Y$ 中包含但 $X$ 中没有的信息。正如我们在前一节中看到的，这分别由 $H(X \mid Y)$ 和 $H(Y \mid X)$ 给出。因此，我们有互信息应为

$$
I(X, Y) = H(X, Y) - H(Y \mid X) - H(X \mid Y).
$$

确实，这是互信息的有效定义。如果我们展开这些术语的定义并将它们组合起来，一点代数运算表明这与

$$I(X, Y) = E_{x} E_{y} \left\{ p_{X, Y}(x, y) \log\frac{p_{X, Y}(x, y)}{p_X(x) p_Y(y)} \right\}. $$
:eqlabel:`eq_mut_ent_def`


我们可以在图 :numref:`fig_mutual_information` 中总结所有这些关系。这是一个很好的直觉测试，看看为什么以下陈述也都等同于 $I(X, Y)$。

* $H(X) - H(X \mid Y)$
* $H(Y) - H(Y \mid X)$
* $H(X) + H(Y) - H(X, Y)$

![互信息与联合熵和条件熵的关系。](../img/mutual-information.svg)
:label:`fig_mutual_information`


在很多方面，我们可以将互信息 :eqref:`eq_mut_ent_def` 视为我们之前在 :numref:`sec_random_variables` 中看到的相关系数的原理扩展。这使我们不仅可以询问变量之间的线性关系，还可以询问两个随机变量之间共享的最大信息量。

现在，让我们从头开始实现互信息。

In [5]:
def mutual_information(p_xy, p_x, p_y):
    p = p_xy / (p_x * p_y)
    mutual = p_xy * torch.log2(p)
    # Operator `nansum` will sum up the non-nan number
    out = nansum(mutual)
    return out

mutual_information(torch.tensor([[0.1, 0.5], [0.1, 0.3]]),
                   torch.tensor([0.2, 0.8]), torch.tensor([[0.75, 0.25]]))

tensor(0.7195)

### 互信息的性质

与其记住互信息的定义 :eqref:`eq_mut_ent_def`，你只需要记住它的显著性质：

* 互信息是对称的，即 $I(X, Y) = I(Y, X)$。
* 互信息是非负的，即 $I(X, Y) \geq 0$。
* 如果且仅当 $X$ 和 $Y$ 独立时，$I(X, Y) = 0$。例如，如果 $X$ 和 $Y$ 独立，那么知道 $Y$ 不会给 $X$ 提供任何信息，反之亦然，因此它们的互信息为零。
* 或者，如果 $X$ 是 $Y$ 的可逆函数，则 $Y$ 和 $X$ 共享所有信息，并且 $$I(X, Y) = H(Y) = H(X)$$。

### 点互信息

在本章开始讨论熵时，我们能够将 $-\log(p_X(x))$ 解释为我们对特定结果的惊讶程度。我们可以对互信息中的对数项进行类似的解释，这通常被称为 *点互信息*：

$$\textrm{pmi}(x, y) = \log\frac{p_{X, Y}(x, y)}{p_X(x) p_Y(y)}.$$
:eqlabel:`eq_pmi_def`

我们可以将 :eqref:`eq_pmi_def` 视为衡量特定结果组合 $x$ 和 $y$ 与独立随机结果相比发生的可能性是多大还是少。如果它很大且为正，则这两个特定结果比随机机会发生得更频繁（注意：分母是 $p_X(x) p_Y(y)$，这是两个结果如果是独立的话的概率），而如果它很大且为负，则表示这两个结果发生的频率远低于随机机会。

这使我们能够将互信息 :eqref:`eq_mut_ent_def` 解释为平均而言，我们看到两个结果一起出现时的惊讶程度与它们独立时的预期相比。

### 互信息的应用

互信息在其纯定义中可能有点抽象，那么它如何与机器学习相关？在自然语言处理中，最困难的问题之一是 *歧义消解*，或者说是词语的意义在上下文中不明确的问题。例如，最近的一条新闻标题报道说“Amazon is on fire”。你可能会想知道是指公司 Amazon 的某栋建筑着火了，还是亚马逊雨林着火了。

在这种情况下，互信息可以帮助我们解决这种歧义。我们首先找到一组每个与公司 Amazon 有较大互信息的词，如电子商务、科技和在线。其次，我们找到另一组每个与亚马逊雨林有较大互信息的词，如雨、森林和热带。当我们需要消除“Amazon”的歧义时，我们可以比较在这两个词组中哪一个在“Amazon”这个词的上下文中出现得更多。在这种情况下，文章会继续描述森林，使上下文变得清晰。

## Kullback–Leibler 散度

正如我们在 :numref:`sec_linear-algebra` 中讨论的那样，我们可以使用范数来测量空间中任意维度两点之间的距离。我们希望对概率分布也能做类似的事情。有很多方法可以实现这一点，但信息论提供了一种很好的方法。我们现在探讨 *Kullback–Leibler (KL) 散度*，它提供了一种衡量两个分布是否接近的方法。

### 定义

给定一个遵循概率分布 $P$ 的随机变量 $X$，其概率密度函数或概率质量函数为 $p(x)$，我们用另一个概率分布 $Q$ 来估计 $P$，其概率密度函数或概率质量函数为 $q(x)$。则 $P$ 和 $Q$ 之间的 *Kullback–Leibler (KL) 散度*（或 *相对熵*）为

$$D_{\textrm{KL}}(P\|Q) = E_{x \sim P} \left[ \log \frac{p(x)}{q(x)} \right].$$
:eqlabel:`eq_kl_def`

与点互信息 :eqref:`eq_pmi_def` 类似，我们可以再次对对数项进行解释：$-\log \frac{q(x)}{p(x)} = -\log(q(x)) - (-\log(p(x)))$ 在 $P$ 下观察到 $x$ 的频率远高于 $Q$ 时会很大且为正，在观察到结果远低于预期时会很大且为负。这样，我们可以将其解释为相对于参考分布观察结果时我们的 *相对* 惊讶程度。

让我们从头实现 KL 散度。

In [6]:
def kl_divergence(p, q):
    kl = p * torch.log2(p / q)
    out = nansum(kl)
    return out.abs().item()

### KL散度的性质

让我们来看看KL散度 :eqref:`eq_kl_def`的一些性质。

* KL散度是非对称的，即存在$P,Q$使得 $$D_{\textrm{KL}}(P\|Q) \neq D_{\textrm{KL}}(Q\|P).$$
* KL散度是非负的，即 $$D_{\textrm{KL}}(P\|Q) \geq 0.$$ 注意，只有当$P = Q$时等式才成立。
* 如果存在某个$x$使得$p(x) > 0$且$q(x) = 0$，则$D_{\textrm{KL}}(P\|Q) = \infty$。
* KL散度与互信息之间有密切关系。除了在:numref:`fig_mutual_information`中显示的关系外，$I(X, Y)$在数值上还等同于以下项：
    1. $D_{\textrm{KL}}(P(X, Y)  \ \| \ P(X)P(Y))$；
    1. $E_Y \{ D_{\textrm{KL}}(P(X \mid Y) \ \| \ P(X)) \}$；
    1. $E_X \{ D_{\textrm{KL}}(P(Y \mid X) \ \| \ P(Y)) \}$。

  对于第一项，我们将互信息解释为$P(X, Y)$和$P(X)$与$P(Y)$乘积之间的KL散度，因此是衡量联合分布与独立分布差异程度的一种度量。对于第二项，互信息告诉我们从得知$X$的分布值后，关于$Y$的不确定性平均减少了多少。第三项类似。


### 示例

让我们通过一个玩具示例来明确看到非对称性。

首先，生成并排序三个长度为$10,000$的张量：一个目标张量$p$，其遵循正态分布$N(0, 1)$；两个候选张量$q_1$和$q_2$，分别遵循正态分布$N(-1, 1)$和$N(1, 1)$。

In [7]:
torch.manual_seed(1)

tensor_len = 10000
p = torch.normal(0, 1, (tensor_len, ))
q1 = torch.normal(-1, 1, (tensor_len, ))
q2 = torch.normal(1, 1, (tensor_len, ))

p = torch.sort(p)[0]
q1 = torch.sort(q1)[0]
q2 = torch.sort(q2)[0]

由于$q_1$和$q_2$关于y轴（即$x=0$）对称，我们期望$D_{\textrm{KL}}(p\|q_1)$和$D_{\textrm{KL}}(p\|q_2)$之间的KL散度值相似。如下所示，$D_{\textrm{KL}}(p\|q_1)$和$D_{\textrm{KL}}(p\|q_2)$之间的差异不到3%。

In [8]:
kl_pq1 = kl_divergence(p, q1)
kl_pq2 = kl_divergence(p, q2)
similar_percentage = abs(kl_pq1 - kl_pq2) / ((kl_pq1 + kl_pq2) / 2) * 100

kl_pq1, kl_pq2, similar_percentage

(8582.0341796875, 8828.3095703125, 2.8290698237936858)

相比之下，你可能会发现$D_{\textrm{KL}}(q_2 \|p)$和$D_{\textrm{KL}}(p \| q_2)$相差很多，如下所示大约相差40%。

In [9]:
kl_q2p = kl_divergence(q2, p)
differ_percentage = abs(kl_q2p - kl_pq2) / ((kl_q2p + kl_pq2) / 2) * 100

kl_q2p, differ_percentage

(14130.125, 46.18621024399691)

## 交叉熵

如果你对信息论在深度学习中的应用感到好奇，这里有一个快速的例子。我们定义真实分布 $P$ 具有概率分布 $p(x)$，估计分布 $Q$ 具有概率分布 $q(x)$，并且在本节的其余部分中将使用它们。

假设我们需要根据给定的 $n$ 个数据示例 {$x_1, \ldots, x_n$} 解决一个二分类问题。假设我们将 $1$ 和 $0$ 分别编码为正类和负类标签 $y_i$，并且我们的神经网络由 $\theta$ 参数化。如果我们旨在找到最佳的 $\theta$ 使得 $\hat{y}_i= p_{\theta}(y_i \mid x_i)$，自然可以应用最大似然方法如 :numref:`sec_maximum_likelihood` 中所见。具体来说，对于真实标签 $y_i$ 和预测 $\hat{y}_i= p_{\theta}(y_i \mid x_i)$，被分类为正类的概率是 $\pi_i= p_{\theta}(y_i = 1 \mid x_i)$。因此，对数似然函数将是

$$
\begin{aligned}
l(\theta) &= \log L(\theta) \\
  &= \log \prod_{i=1}^n \pi_i^{y_i} (1 - \pi_i)^{1 - y_i} \\
  &= \sum_{i=1}^n y_i \log(\pi_i) + (1 - y_i) \log (1 - \pi_i). \\
\end{aligned}
$$

最大化对数似然函数 $l(\theta)$ 等同于最小化 $- l(\theta)$，因此我们可以从这里找到最佳的 $\theta$。为了将上述损失推广到任何分布，我们也称 $-l(\theta)$ 为 *交叉熵损失* $\textrm{CE}(y, \hat{y})$，其中 $y$ 遵循真实分布 $P$ 而 $\hat{y}$ 遵循估计分布 $Q$。

这一切都是从最大似然的角度推导出来的。然而，如果我们仔细观察，可以看到像 $\log(\pi_i)$ 这样的项已经进入了我们的计算，这明确表明我们可以从信息论的角度来理解这个表达式。

### 形式定义

与KL散度一样，对于随机变量 $X$，我们也可以通过 *交叉熵* 来测量估计分布 $Q$ 和真实分布 $P$ 之间的差异，

$$\textrm{CE}(P, Q) = - E_{x \sim P} [\log(q(x))].$$
:eqlabel:`eq_ce_def`

利用上面讨论的熵性质，我们还可以将其解释为熵 $H(P)$ 和 $P$ 与 $Q$ 之间的KL散度之和，即

$$\textrm{CE} (P, Q) = H(P) + D_{\textrm{KL}}(P\|Q).$$

我们可以如下实现交叉熵损失。

In [10]:
def cross_entropy(y_hat, y):
    ce = -torch.log(y_hat[range(len(y_hat)), y])
    return ce.mean()

现在定义两个张量分别表示标签和预测值，并计算它们的交叉熵损失。

In [11]:
labels = torch.tensor([0, 2])
preds = torch.tensor([[0.3, 0.6, 0.1], [0.2, 0.3, 0.5]])

cross_entropy(preds, labels)

tensor(0.9486)

### 属性

正如本节开头所暗示的，交叉熵 :eqref:`eq_ce_def` 可以用于定义优化问题中的损失函数。事实证明，以下几点是等价的：

1. 最大化分布 $P$ 对于 $Q$ 的预测概率（即 $E_{x \sim P} [\log (q(x))]$）；
1. 最小化交叉熵 $\textrm{CE} (P, Q)$；
1. 最小化 KL 散度 $D_{\textrm{KL}}(P\|Q)$。

交叉熵的定义间接证明了目标2和目标3之间的等价关系，只要真实数据的熵 $H(P)$ 是常数。


### 交叉熵作为多分类的目标函数

如果我们深入研究带有交叉熵损失 $\textrm{CE}$ 的分类目标函数，我们会发现最小化 $\textrm{CE}$ 等同于最大化对数似然函数 $L$。

首先，假设我们有一个包含 $n$ 个样本的数据集，并且它可以被分为 $k$ 类。对于每个数据样本 $i$，我们使用 *独热编码* 表示任何 $k$ 类标签 $\mathbf{y}_i = (y_{i1}, \ldots, y_{ik})$。具体来说，如果样本 $i$ 属于类 $j$，则我们将第 $j$ 项设置为 $1$，所有其他分量设置为 $0$，即

$$ y_{ij} = \begin{cases}1 & j \in J; \\ 0 &\textrm{否则。}\end{cases}$$

例如，如果一个多分类问题包含三个类 $A$、$B$ 和 $C$，那么标签 $\mathbf{y}_i$ 可以编码为 {$A: (1, 0, 0); B: (0, 1, 0); C: (0, 0, 1)$}。

假设我们的神经网络由 $\theta$ 参数化。对于真实标签向量 $\mathbf{y}_i$ 和预测 $$\hat{\mathbf{y}}_i= p_{\theta}(\mathbf{y}_i \mid \mathbf{x}_i) = \sum_{j=1}^k y_{ij} p_{\theta} (y_{ij}  \mid  \mathbf{x}_i).$$

因此，*交叉熵损失* 将是

$$
\textrm{CE}(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{i=1}^n \mathbf{y}_i \log \hat{\mathbf{y}}_i
 = - \sum_{i=1}^n \sum_{j=1}^k y_{ij} \log{p_{\theta} (y_{ij}  \mid  \mathbf{x}_i)}.\\
$$

另一方面，我们也可以通过最大似然估计来解决这个问题。首先，让我们快速介绍一个 $k$ 类多项式分布。它是从二元类扩展到多类的伯努利分布。如果随机变量 $\mathbf{z} = (z_{1}, \ldots, z_{k})$ 遵循具有概率 $\mathbf{p} =$ ($p_{1}, \ldots, p_{k}$) 的 $k$ 类 *多项式分布*，即 $$p(\mathbf{z}) = p(z_1, \ldots, z_k) = \textrm{Multi} (p_1, \ldots, p_k), \textrm{ 其中 } \sum_{i=1}^k p_i = 1,$$ 则 $\mathbf{z}$ 的联合概率质量函数(p.m.f.) 为
$$\mathbf{p}^\mathbf{z} = \prod_{j=1}^k p_{j}^{z_{j}}.$$


可以看出，每个数据样本的标签 $\mathbf{y}_i$ 遵循具有概率 $\boldsymbol{\pi} =$ ($\pi_{1}, \ldots, \pi_{k}$) 的 $k$ 类多项式分布。因此，每个数据样本 $\mathbf{y}_i$ 的联合 p.m.f. 为 $\mathbf{\pi}^{\mathbf{y}_i} = \prod_{j=1}^k \pi_{j}^{y_{ij}}.$
因此，对数似然函数将是

$$
\begin{aligned}
l(\theta)
 = \log L(\theta)
 = \log \prod_{i=1}^n \boldsymbol{\pi}^{\mathbf{y}_i}
 = \log \prod_{i=1}^n \prod_{j=1}^k \pi_{j}^{y_{ij}}
 = \sum_{i=1}^n \sum_{j=1}^k y_{ij} \log{\pi_{j}}.\\
\end{aligned}
$$

因为在最大似然估计中，我们通过使 $\pi_{j} = p_{\theta} (y_{ij}  \mid  \mathbf{x}_i)$ 来最大化目标函数 $l(\theta)$。因此，对于任何多分类，最大化上述对数似然函数 $l(\theta)$ 等同于最小化 CE 损失 $\textrm{CE}(y, \hat{y})$。


为了测试上述证明，让我们应用内置的度量 `NegativeLogLikelihood`。使用与前面示例相同的 `labels` 和 `preds`，我们将得到与前一个示例相同的数值损失，精确到小数点后5位。

In [12]:
# Implementation of cross-entropy loss in PyTorch combines `nn.LogSoftmax()`
# and `nn.NLLLoss()`
nll_loss = NLLLoss()
loss = nll_loss(torch.log(preds), labels)
loss

tensor(0.9486)

## 摘要

* 信息论是关于编码、解码、传输和处理信息的研究领域。
* 熵是衡量不同信号中包含多少信息的单位。
* KL散度也可以用来衡量两个分布之间的差异。
* 交叉熵可以看作是多类分类的目标函数。最小化交叉熵损失等同于最大化对数似然函数。

## 练习

1. 验证第一部分中的卡片示例确实具有所声称的熵。
1. 证明对于所有分布 $p$ 和 $q$，KL散度 $D(p\|q)$ 是非负的。提示：使用Jensen不等式，即利用 $-\log x$ 是一个凸函数的事实。
1. 让我们从几个数据源计算熵：
    * 假设你正在观看一只猴子在打字机上产生的输出。这只猴子随机按下打字机上的任意44个键（你可以假设它还没有发现任何特殊键或换档键）。每字符你能观察到多少比特的随机性？
    * 对猴子的表现不满意，你用一个醉酒的排字员替换了它。它可以生成单词，尽管不是连贯的。相反，它从2000个词汇表中随机挑选一个单词。假设英语中平均每个单词的长度为4.5个字母。现在每字符你能观察到多少比特的随机性？
    * 对结果仍然不满意，你用一个高质量的语言模型替换了排字员。该语言模型目前可以获得最低达到每词15点的困惑度。字符困惑度定义为一组概率的几何平均值的倒数，每个概率对应单词中的一个字符。具体来说，如果给定单词的长度为$l$，那么$\textrm{PPL}(\textrm{word}) = \left[\prod_i p(\textrm{character}_i)\right]^{ -\frac{1}{l}} = \exp \left[ - \frac{1}{l} \sum_i{\log p(\textrm{character}_i)} \right].$ 假设测试单词有4.5个字母，现在每字符你能观察到多少比特的随机性？
1. 直观地解释为什么$I(X, Y) = H(X) - H(X \mid Y)$。然后，通过将两边表示为关于联合分布的期望来证明这一点。
1. 两个高斯分布 $\mathcal{N}(\mu_1, \sigma_1^2)$ 和 $\mathcal{N}(\mu_2, \sigma_2^2)$ 之间的KL散度是什么？

[讨论](https://discuss.d2l.ai/t/1104)